# Snuffelfiets kwartaalrapportages

## Settings.

In [ ]:
# Generieke imports, variabelen en functies.

from pathlib import Path

import numpy as np
import pandas as pd

from snuffelfiets import inlezen, opschonen, analyse, plotting

api_key = ''  # voeg hier de API key toe

data_directory = Path('~','kwartaalrapportage').expanduser()
prefix = 'api_gegevens'

# date range selection settings
quarter = 'Q1'
year = 2023
quarters = {
    'Q1': [1, 2, 3], 
    'Q2': [4, 5, 6], 
    'Q3': [7, 8, 9], 
    'Q4': [10, 11, 12], 
}
yq = f'{year}_{quarter}'

# data processing settings
error_code_selection = []
rit_splitter_interval = 1800  # s
ritfilters = dict(
    min_measurements=2,     # #
    max_duration=360,       # minutes
    max_distance=200,       # kilometers
    min_average_speed=1,    # km/h
    max_average_speed=35,   # km/h
    )
threshold_pm2_5 = 100

# mapbox settings (in degrees latlon)
b = {
    'N': [52.303634, 5.013507],
    'Z':  [51.858631, 5.040462],
    'O':  [51.954780, 5.627990],
    'W':  [52.226808, 4.794457],
}
mapbox_center = {
    'lat': b['Z'][0] + 0.5 * (b['N'][0] - b['Z'][0]),
    'lon': b['W'][1] + 0.5 * (b['O'][1] - b['W'][1]),
}
mapbox_extent = 1
hexagon_size = 0.010
hexbin_args = {
    'agg_func': np.nanmean,
    'color_continuous_scale': plotting.discrete_colorscale(),
    'range_color': [0, threshold_pm2_5],
    'min_count': 2,
    'animation_frame': None,
    'width': 1920,
    'height': 1080,
    'opacity': 1.0,
    'zoom': 10,
    'center': mapbox_center,
    }
# add Utrecht polygons
# Import Utrecht province and township polygons
filepaths = plotting.download_borders_utrecht(data_directory)
provincies, gemeenten = plotting.get_borders_utrecht(data_directory, *filepaths)
mapbox_layers = [{
    "name": "Gemeenten",
    "below": 'traces',
    "sourcetype": "geojson",
    "type": "line",
    "color": "gray",
    "source": gemeenten,
    }, 
    {
    "name": "Provincies",
    "below": 'traces',
    "sourcetype": "geojson",
    "type": "line",
    "color": "red",
    "source": provincies,
    }]
layout_args = {
    'mapbox_style': 'open-street-map',
    'coloraxis_showscale': False,
    'mapbox_layers': mapbox_layers,
    }

# Directories
output_directory = Path(data_directory, yq)
output_directory.mkdir(parents=True, exist_ok=True)

print(f'Analysing quarter {yq}; writing output to {output_directory}.')


## Read data with API and save as monthly CSV.

This data is uploaded to the Google Drive of the MCU dataclub:
Dataclub MCU > Project Snuffelfiets > data > api_gegevens_{????}-{??}.csv

In [ ]:
years = [2024]  # [2021, 2022, 2023]
months = range(2, 3)  # range(1, 13)

for y in years:
    for m in months:
        inlezen.monthly_csv_dump(api_key, y, m, data_directory, prefix)


## Read data from monthly CSV's.


In [ ]:
# Read data from saved csvs.
dfs = []
for m in quarters[quarter]:
    filename = f'{prefix}_{year}-{m:02d}.csv'
    p = Path(data_directory, filename)
    df = pd.read_csv(p)
    dfs.append(df)
df = pd.concat(dfs)

print(f'Read {df.shape[0]} measurements.')


## Data preparation

In [ ]:
# Get some insight in the error modes present in the dataset.
opschonen.analyse_errors(df)


In [ ]:
# Drop the errors.
df = opschonen.verwijder_errors(df, error_codes=error_code_selection)


In [ ]:
# Convert timestamps to datetime objects and add dt columns.
df = analyse.bewerk_timestamp(df, split=True)


In [ ]:
# Split measurements into rides and add cycle stat columns.
df = analyse.split_in_ritten(df, t_seconden=rit_splitter_interval)


In [ ]:
# Filter the rides.
df = analyse.filter_ritten(df, **ritfilters)


## Summary Snuffelfiets statistics.

In [ ]:
def printfun(period, sumstats):

    print(f'\n==== {period} ====\n')

    print(f"Aantal fietsers: {sumstats['fietsers']['N']}\n")

    print(f"{' ':20} {'totaal':12} {'gemiddeld':12} {'topper':12}")
    print(f'-' * 56)
    stat = 'uren'
    print(f"FIETSTIJD [uur]:  {sumstats[stat]['N']:12f} {sumstats[stat]['G']:12f} {sumstats[stat]['M']:12f}")
    stat = 'afstand'
    print(f"AFSTAND    [km]:  {sumstats[stat]['N']:12f} {sumstats[stat]['G']:12f} {sumstats[stat]['M']:12f}")
    stat = 'ritten'
    print(f"RITTEN      [#]:  {sumstats[stat]['N']:12f} {sumstats[stat]['G']:12f} {sumstats[stat]['M']:12f}")


In [ ]:
# Print the summary statistics for the quarter.
sumstats = analyse.summary_stats(df)
printfun(quarter, sumstats)


In [ ]:
# Print the summary statistics for the months in the quarter.
for m, dfm in df.groupby('month'):
    sumstats = analyse.summary_stats(dfm)
    printfun(f'{year}{m:02d}', sumstats)


## Air quality evaluation

In [ ]:
# Plot categorical hist of PM2.5 values.
def plot_hbar_cat(df, bins, labels, title=''):
    df[f'{var}_cat'] = pd.cut(df[var], bins=bins, labels=labels)
    ax = df[['entity_id', f'{var}_cat']].groupby(f'{var}_cat', observed=False).count().plot.barh(stacked=True, legend=False)
    ax.invert_yaxis()
    ax.axes.get_yaxis().get_label().set_visible(False)
    ax.axes.get_xaxis().set_label_text("Measurement count")
    ax.set_title(title)

var = 'pm2_5'
bins = [0, 1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
labels = [0, 1, 2, 5, 10, 20, 50, 100, 200, 500]
plot_hbar_cat(df, bins, labels, title='PM2.5')


In [ ]:
# Limit the PM2.5 values. FIXME: we need a better and validated method
df['pm2_5'][df['pm2_5'] >= threshold_pm2_5] = np.nan

# Plot the histogram.
df['pm2_5'].hist(bins=200, grid=False)


In [ ]:
## Hexbin plots of PM2.5

# Remove datapoints outside of the map area 
#   because it would take a very long time to process large areas.
#   TODO: doe dit op ritniveau (verwijderen ritten deels of geheel buiten de target area)
latlon = {
    'latitude': {'center': mapbox_center['lat'], 'extent': mapbox_extent},
    'longitude': {'center': mapbox_center['lon'], 'extent': mapbox_extent},
}
df = opschonen.filter_lat_lon(df, latlon)

# Plot the data for each month.
for m, dfm in df.groupby('month'):

    yyyymm = f'{year}{m:02d}'
    hexbin_args['title'] = yyyymm
    fig = plotting.hexbin_mapbox(dfm, hexagon_size, hexbin_args, layout_args)

    # Save image
    filestem = f'{yyyymm}_hexbin'
    output_stem = Path(output_directory, filestem)
    fig.write_html(f"{output_stem}.html")
    fig.write_image(f"{output_stem}.png")
    fig.write_image(f"{output_stem}.pdf")


## Uitgelicht 2023-Q1: Fijnstof en regen.

In [ ]:
# Specify the date range.
import calendar
months = quarters[quarter]
lastday = calendar.monthrange(year, quarters[quarter][2])[1]
dt_min = f'{year}-{months[0]}-1 00:00:00'
dt_max = f'{year}-{months[2]}-{lastday} 23:59:59'

# Import the weather data.
dfr = analyse.import_knmi_data(dt_min, dt_max, interval='dag', stations=[260], variables=['RH'])
dfr.RH = dfr.RH.values / 10  # RH: Etmaalsom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm)


In [ ]:
# Print the monthly pm2.5 average.
df.loc[: , ['month', 'pm2_5']].groupby('month').mean()


In [ ]:
# Print the monthly rainfall sum.
dfr['month'] = dfr.index.month
dfr.loc[: , ['month', 'RH']].groupby('month').sum()


In [ ]:
# Calculate the mean daily PM2.5 value.
sfday = df.loc[:, ['pm2_5', 'day', 'month', 'year']].groupby(['day', 'month', 'year']).mean()
# Add to the weather data frame.
dfr['pm2_5'] = np.array(sfday['pm2_5'])


In [ ]:
# Plot the data.
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots(dpi=300)
ax2 = ax1.twinx()

ax1.plot(dfr.index, dfr.pm2_5, c='C1')
ax2.plot(dfr.index, dfr.RH, c='C0')

ax1.set_ylabel('PM2.5 [ug/m3]', color='C1')
ax2.set_ylabel('Etmaalsom van de neerslag [mm]', color='C0')

ax1.plot([dfr.index[0], dfr.index[-1]], [25, 25], c='C1',linestyle='--')
plt.text(0.86, 0.56, 'norm', c='C1', transform=ax1.transAxes)
ax1.xaxis.set_tick_params(rotation=45)
plt.grid(linestyle=':')
